In [3]:
#Librerías
import pandas as pd

# MMM Ventas

## Tabla Final

A continuación cargamos todas las tablas que vamos a emplear para el modelo de atribución:

In [4]:
df_web = pd.read_csv("../data/raw/web.csv", sep=",")
df_offline = pd.read_csv("../data/raw/offline.csv", sep=",")
df_time = pd.read_csv("../data/raw/time.csv", sep=",")
df_visit = pd.read_csv("../data/raw/visit.csv", sep=",")
df_inv = pd.read_csv("../data/raw/inv.csv", sep=",")

Tenemos que añadir los sufijos web y offline a las columans de estas tablas ya que sus nombre coinciden y posteriormente al hacer el merge necesitamos distinguirlas.

In [5]:
df_offline = df_offline.rename(columns={
    col: f"{col}_off" for col in df_offline.columns if col != "ID_Date"})


df_visit = df_visit.rename(columns={
    col: f"{col}_visit" for col in df_visit.columns if col != "ID_Date"})


**Tabla Inversión**

In [6]:
df_inv = pd.read_csv("../data/raw/inv.csv", sep=",")
df_inv.head()

,ID_Date,CINE,EXTERIOR,INTERNET,PRENSA,PRODUCCION,RADIO,REVISTAS,PlataformasVideo,VARIOS,INV_Total
0,202201,59887.84,8200.00,133666.43,0.0,6050.0,2426.40,18013.50,338824.94,7531.03,574600.14
1,202202,72291.56,94627.51,189275.94,10000.0,6700.0,0.00,49110.91,487537.14,6972.02,916515.08
2,202203,55000.00,0.00,214037.62,4500.0,3700.0,0.00,0.00,75000.00,15284.85,367522.47
3,202204,10883.30,0.00,194816.97,48800.0,21550.0,1140.00,40876.00,610865.89,12255.75,941187.91
4,202205,74333.20,0.00,130795.16,11000.0,4900.0,102851.86,20775.75,393807.74,7926.46,746390.17


Se han generado las siguientes métricas a partir de la tabla de inversión mensual(de cara a facilitar la evaluación del budget):

- `Pct_Online`: porcentaje de la inversión total del mes destinada a medios online.
- `Pct_Offline`: porcentaje de la inversión total del mes destinada a medios offline.

In [7]:

# online
inversion_online = df_inv['INTERNET'] + df_inv['PlataformasVideo']

# offline
inversion_offline = (
    df_inv['CINE'] + df_inv['EXTERIOR'] + df_inv['PRENSA'] + df_inv['RADIO'] +
    df_inv['REVISTAS'] + df_inv['PRODUCCION'] + df_inv['VARIOS']
)

# porcentaje de cada tipo de inv/total_inv
df_inv['Pct_Online'] = inversion_online / df_inv['INV_Total']
df_inv['Pct_Offline'] = inversion_offline / df_inv['INV_Total']


In [8]:
df_inv.head()

,ID_Date,CINE,EXTERIOR,INTERNET,PRENSA,PRODUCCION,RADIO,REVISTAS,PlataformasVideo,VARIOS,INV_Total,Pct_Online,Pct_Offline
0,202201,59887.84,8200.00,133666.43,0.0,6050.0,2426.40,18013.50,338824.94,7531.03,574600.14,0.822296,0.177704
1,202202,72291.56,94627.51,189275.94,10000.0,6700.0,0.00,49110.91,487537.14,6972.02,916515.08,0.738464,0.261536
2,202203,55000.00,0.00,214037.62,4500.0,3700.0,0.00,0.00,75000.00,15284.85,367522.47,0.786449,0.213551
3,202204,10883.30,0.00,194816.97,48800.0,21550.0,1140.00,40876.00,610865.89,12255.75,941187.91,0.856028,0.143972
4,202205,74333.20,0.00,130795.16,11000.0,4900.0,102851.86,20775.75,393807.74,7926.46,746390.17,0.702853,0.297147


**Merge para Tabla Final**

In [9]:
# Union todas las tablas por PK=ID_date
df_merge = df_inv.merge(df_web, on='ID_Date')
df_merge = df_merge.merge(df_offline, on='ID_Date')
df_merge = df_merge.merge(df_time, on='ID_Date')
df_merge = df_merge.merge(df_visit, on='ID_Date')
df_merge = df_merge.sort_values('ID_Date')

# Calculamos el delay de compras
df_merge['Sales_1_ago'] = df_merge['Sales_off'].shift(1) # 1 mes atras
df_merge['Sales_2_ago'] = df_merge['Sales_off'].shift(2) # 2 meses atras


*Nota*:Para los primeros dos meses deconcoemos los visitantes previos pero los dejamos como NaN para que lo gestione el modelo.

In [10]:
df_merge.head()

,ID_Date,CINE,EXTERIOR,INTERNET,PRENSA,PRODUCCION,RADIO,REVISTAS,PlataformasVideo,VARIOS,...,RopaHombre_visit,RopaMujer_visit,Complementos_visit,Zapatos_visit,Home_visit,Interior_visit,Otros_visit,SR_Total_visit,Sales_1_ago,Sales_2_ago
0,202201,59887.84,8200.00,133666.43,0.0,6050.0,2426.40,18013.50,338824.94,7531.03,...,50140,81289,16190,36790,18848,9906,183,213346,NaN,NaN
1,202202,72291.56,94627.51,189275.94,10000.0,6700.0,0.00,49110.91,487537.14,6972.02,...,51217,152805,26083,37960,20047,11057,461,299630,1299.0,NaN
2,202203,55000.00,0.00,214037.62,4500.0,3700.0,0.00,0.00,75000.00,15284.85,...,49163,68383,18109,37765,20105,66200,198,259923,1752.0,1299.0
3,202204,10883.30,0.00,194816.97,48800.0,21550.0,1140.00,40876.00,610865.89,12255.75,...,48219,65126,16985,76244,17629,17827,176,242206,1897.0,1752.0
4,202205,74333.20,0.00,130795.16,11000.0,4900.0,102851.86,20775.75,393807.74,7926.46,...,42229,44577,16498,79434,18526,10738,186,212188,1546.0,1897.0


In [11]:
df_merge.columns

Index(['ID_Date', 'CINE', 'EXTERIOR', 'INTERNET', 'PRENSA', 'PRODUCCION',
       'RADIO', 'REVISTAS', 'PlataformasVideo', 'VARIOS', 'INV_Total',
       'Pct_Online', 'Pct_Offline', 'Unique_visitors',
       'PDFBrochuresDownloaded', 'ProductConfigurator',
       'Product_configurator_Visists', 'SocialNetworks', 'DirectTraffic',
       'EMail', 'NaturalSearch', 'OnlineMedia', 'OtherReferrer', 'PaidSearch',
       'Visit_Store_off', 'Mercado_off', 'Sales_off', 'Complementos_off',
       'Ropa_hombre_off', 'Zapatos_off', 'Ropa_Mujer_off', 'Home_off',
       'Interior_off', 'Otros_off', 'Ticket_medio_off', 'Dias_mes',
       'Dia_inicio_mes', 'Dia_findemes', 'working_days', 'Dias_fines_semana',
       'Easterweek', 'RopaHombre_visit', 'RopaMujer_visit',
       'Complementos_visit', 'Zapatos_visit', 'Home_visit', 'Interior_visit',
       'Otros_visit', 'SR_Total_visit', 'Sales_1_ago', 'Sales_2_ago'],
      dtype='object')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcula la matriz de correlación solo con numéricas
cor_matrix = df_merge.corr(numeric_only=True)

# Filtra correlaciones con Sales_off
cor_sales = cor_matrix['Sales_off'].sort_values(ascending=False)

# Visualiza las top 15 más correlacionadas
plt.figure(figsize=(10, 12))
sns.heatmap(cor_sales.head(15).to_frame(), annot=True, cmap='coolwarm')
plt.title("Top correlaciones con Sales_off")
plt.show()
